In [1]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00


In [13]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 40.2 MB/s eta 0:00:00


In [2]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.69.0
    Uninstalling openai-1.69.0:
      Successfully uninstalled openai-1.69.0


In [7]:
import openai
import os
import re
import requests
from PyPDF2 import PdfReader
import pandas as pd
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Increase the max_length limit
nlp.max_length = 2000000
os.environ['OPENAI_API_KEY'] = "sk-svcacct-38lVsQJmAnpHYYGlwHLRfemYH6XipJ83Bn4AHnE19lo8NioSbNQshLh_0Ie6gFHT3BlbkFJgi4LuRlCOMye2qnlGRAGY4_JQCoLousyuBpUsrPte0HftMthAZWRdreWw1EFRdQA"
openai.api_key = os.environ["OPENAI_API_KEY"]
pdf_urls = [
    "https://github.com/engie4800/dsi-capstone-spring-2025-JnJ/raw/main/sampledocs/committee-papers-Anhydrous%20sodium%20thiosulfate-Recommend.pdf",
    "https://github.com/engie4800/dsi-capstone-spring-2025-JnJ/raw/main/sampledocs/committee-papers-Baricitinib-not%20recommend.pdf",
    "https://github.com/engie4800/dsi-capstone-spring-2025-JnJ/raw/main/sampledocs/committee-papers-Bevacizumab%20gamma-recommend.pdf",
    "https://github.com/engie4800/dsi-capstone-spring-2025-JnJ/raw/main/sampledocs/committee-papers-Dupilumab-not%20recommend.pdf",
    "https://github.com/engie4800/dsi-capstone-spring-2025-JnJ/raw/main/sampledocs/committee-papers-Lorlatinib-not%20recommend.pdf",
    "https://github.com/engie4800/dsi-capstone-spring-2025-JnJ/raw/main/sampledocs/committee-papers-Lu%20vipivotide-not%20recommend.pdf",
    "https://github.com/engie4800/dsi-capstone-spring-2025-JnJ/raw/main/sampledocs/committee-papers-Pembrolizumab-recommend.pdf",
    "https://github.com/engie4800/dsi-capstone-spring-2025-JnJ/raw/main/sampledocs/committee-papers-Tixagevimab-not%20recommend.pdf",
    "https://github.com/engie4800/dsi-capstone-spring-2025-JnJ/raw/main/sampledocs/committee-papers-crizotinib-recommend.pdf",
    "https://github.com/engie4800/dsi-capstone-spring-2025-JnJ/raw/main/sampledocs/committee-papers-eplontersen-recommend.pdf"
]

def download_file(url, local_filename):
    # Download the file from `url` and save it locally under `local_filename`
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    return local_filename

# Read PDF
def read_pdf(filepath):
    with open(filepath, "rb") as file:
        reader = PdfReader(file)
        return " ".join([page.extract_text() for page in reader.pages if page.extract_text()])



In [41]:
import pdfplumber
import re


def extract_adverse_reactions_section(pdf_path, start_page=20):
    """
    Extracts the 'Adverse reactions' section from a PDF, starting from the given page,
    and continuing until the next subheader (B.x, B.x.x, Bx, Bx.x) or end of file.

    This version works whether or not the section is prefixed with something like 'B.2.1'.

    :param pdf_path: Path to the PDF file
    :param start_page: 1-based index of the page to begin scanning
    :return: Extracted text content from the 'Adverse reactions' section
    """
    try:
        # Read all text starting from the start_page
        pages_text = []
        with pdfplumber.open(pdf_path) as pdf:
            for page_index in range(start_page - 1, len(pdf.pages)):
                page_text = pdf.pages[page_index].extract_text() or ""
                pages_text.append(page_text)

        full_text = "\n".join(pages_text)

        # More flexible pattern:
        pattern = (
            r"(?s)"  # DOTALL to match across lines
            r"(?:B[\.\d]*\s*)?"  # Optional section label like B.2.1 or B3.1
            r"(Adverse reactions.*?)"  # Capture from 'Adverse reactions'
            r"(?=\n(?:B\.\d+(?:\.\d+)?|B\d+(?:\.\d+)?|\Z))"  # Stop at next section header or EOF
        )

        match = re.search(pattern, full_text, re.IGNORECASE)
        if match:
            return match.group(1).strip()
        else:
            print("[WARN] 'Adverse reactions' section not found.")
            return None

    except Exception as e:
        print(f"[ERROR] Failed to process {pdf_path}: {e}")
        return None



def summarize_adverse_reactions(paragraphs):
    if not paragraphs:
        return "NA"
    prompt = (
        "From the text below, extract only the types of adverse reactions mentioned. AEs means adverse events in context"
        "The context talks about the adverse reactions in different study groups"
        "check the adverse reactions in each study groups and rank it by checking if those study groups have similar adverse reactions or not"
        "Then rank them on a scale from 1 to 10, 1 is context mentioned that study groups have similar adverse reactions and are not that severe; 10 is context mentioned that some study groups have high adverse reactions or adverse events than other study groups (more than 12% higher of some adverse events or adverse reactions in one groups than another study group). If no percentage number mentioned, look the similarity of adverse events (rank 5 means some study groups have 2 or less high adverse reactions or adverse events than other study groups ) and severity and give a ranking number "
        "You must return a rank. Return only one rank number without sentences before the rank numbe like The adverse reactions mentioned in the text include.\n\n"
        + ("\n".join(paragraphs) if isinstance(paragraphs, list) else paragraphs)
    )
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=10,
        temperature=0,
    )
    return response.choices[0].message.content.strip()


In [42]:
import os

data = []

for url in pdf_urls:
    try:
        filename = os.path.basename(url)
        local_filepath = download_file(url, filename)
        print(f"Processing: {filename}")

        # Pass the PDF path, not the text!
        adverse_section = extract_adverse_reactions_section(local_filepath)
        severity_rank = summarize_adverse_reactions(adverse_section)

        row = {
            "Filename": filename,
            "Adverse Reaction Severity": severity_rank
        }
        data.append(row)

    except Exception as e:
        print(f"Failed to process {filename}: {e}")


Processing: committee-papers-Anhydrous%20sodium%20thiosulfate-Recommend.pdf
Processing: committee-papers-Baricitinib-not%20recommend.pdf
Processing: committee-papers-Bevacizumab%20gamma-recommend.pdf
Processing: committee-papers-Dupilumab-not%20recommend.pdf
Processing: committee-papers-Lorlatinib-not%20recommend.pdf
Processing: committee-papers-Lu%20vipivotide-not%20recommend.pdf
Processing: committee-papers-Pembrolizumab-recommend.pdf
Processing: committee-papers-Tixagevimab-not%20recommend.pdf
Processing: committee-papers-crizotinib-recommend.pdf
Processing: committee-papers-eplontersen-recommend.pdf


In [24]:
adverse_section

'Adverse reactions\nHELIOS-A was the pivotal study assessing the safety and efficacy of vutrisiran.12 To illustrate the\ncomparable safety profiles of eplontersen and vutrisiran, safety data have been presented from\nNEURO-TTRansform (including the external placebo arm from NEURO-TTR) and HELIOS-A.\nWhere safety data were collected at multiple timepoints, data with the longest follow-up have\nbeen presented since they are likely to be the most representative. Conversely, efficacy data\nwere presented at Week 66 in this appraisal, as efficacy endpoints at Week 85 were exploratory\nand assessed as a post hoc outcome. Additionally, efficacy data for the NEURO-TTR external\nplacebo group were not collected beyond Week 66.\nTreatment-Emergent Adverse Events\nAn overview of the treatment-emergent adverse events (TEAEs) from the final analysis of\nNEURO-TTRansform and HELIOS-A is presented in Table 36. In NEURO-TTRansform, the final\nanalysis of safety data took place at Week 85 for the eplon

In [40]:
df = pd.DataFrame(data)
df

,Filename,Adverse Reaction Severity
0,committee-papers-Anhydrous%20sodium%20thiosulf...,1
1,committee-papers-Baricitinib-not%20recommend.pdf,1
2,committee-papers-Bevacizumab%20gamma-recommend...,3
3,committee-papers-Dupilumab-not%20recommend.pdf,5
4,committee-papers-Lorlatinib-not%20recommend.pdf,5
5,committee-papers-Lu%20vipivotide-not%20recomme...,9
6,committee-papers-Pembrolizumab-recommend.pdf,9
7,committee-papers-Tixagevimab-not%20recommend.pdf,5
8,committee-papers-crizotinib-recommend.pdf,7
9,committee-papers-eplontersen-recommend.pdf,5


In [34]:
df.to_csv("10_adverse.csv")